# QARTOD - NetCDF Examples

This notebook provides examples of running QARTOD on a netCDF file. For background, see [XarrayStream](https://ioos.github.io/ioos_qc/usage.html#xarraystream) and [CFNetCDFStore](https://ioos.github.io/ioos_qc/usage.html#cfnetcdfstore) in the docs.

There are multiple ways that you can integrate `ioos_qc` into your netcdf-based workflow. 

**Option A:** Store test configurations externally, pass your configuration and netcdf file to `ioos_qc`, and manually update netcdf variables with results of the test
  * In this case, you extract variables from the netcdf file, use `ioos_qc` methods to run tests, and then manually update the netcdf file with results
  * This provides the most control, but doesn't take advantage of shared code in the `ioos_qc` library
  * It's up to you to ensure your resulting netcdf is self-describing and CF-compliant

**Option B:** Store test configurations externally, then pass your configuration and netcdf file to `ioos_qc`, and let it run tests and update the file with results
  * This takes advantage of `ioos_qc` code to store results and configuration in the netCDF file, and ensure a self-describing, CF-compliant file
  * Managing your test configurations outside the file is better when dealing with a large number of datasets/configurations

**Option C:** Store test configurations in your netcdf file, then pass that file to `ioos_qc` and let it run tests and update the file with results
  * You only need to add test configurations to the file one time, and after that you could run tests over and over again on the same file
  * This option is the most portable, since the data, configuration, and results are all in one place
  * The downside is, test configuration management is difficult since it's stored in the file instead of some common external location
  
  


## Setup

In [ ]:
from bokeh.plotting import output_notebook

output_notebook()

In [ ]:
def plot_ncresults(ncdata, var_name, results, title, test_name):
    """Helper method to plot QC results using Bokeh."""
    
    qc_test = next(r for r in results if r.stream_id == var_name and r.test == test_name)
    time = np.array(qc_test.tinp)
    obs = np.array(qc_test.data)
    results = qc_test.results

    qc_pass = np.ma.masked_where(results != 1, obs)
    num_pass = (results == 1).sum()
    qc_suspect = np.ma.masked_where(results != 3, obs)
    num_suspect = (results == 3).sum()
    qc_fail = np.ma.masked_where(results != 4, obs)
    num_fail = (results == 4).sum()
    qc_notrun = np.ma.masked_where(results != 2, obs)

    p1 = figure(
        x_axis_type="datetime",
        title=f"{test_name} : {title} : p/s/f= {num_pass}/{num_suspect}/{num_fail}")
    p1.grid.grid_line_alpha = 0.3
    p1.xaxis.axis_label = "Time"
    p1.yaxis.axis_label = "Observation Value"
    
    p1.line(time, obs, legend_label="obs", color="#A6CEE3")
    p1.circle(
        time, qc_notrun, size=2, legend_label="qc not run", color="gray", alpha=0.2
    )
    p1.circle(time, qc_pass, size=4, legend_label="qc pass", color="green", alpha=0.5)
    p1.circle(
        time, qc_suspect, size=4, legend_label="qc suspect", color="orange", alpha=0.7
    )
    p1.circle(time, qc_fail, size=6, legend_label="qc fail", color="red", alpha=1.0)

    show(gridplot([[p1]], width=800, height=400))

In [ ]:
import os
import shutil
import tempfile
from datetime import datetime
import pandas as pd

## Load the netCDF dataset


The example netCDF dataset is a pCO2 sensor from the Ocean Observatories Initiative (OOI) Coastal Endurance Inshore Surface Mooring instrument frame at 7 meters depth located on the Oregon Shelf break.


In [ ]:
import xarray as xr
from erddapy.url_handling import urlopen
from netCDF4 import Dataset


def open_from_https(url):

    data = urlopen(fname)
    nc = Dataset("pco2_netcdf_example", memory=data.read())
    return xr.open_dataset(xr.backends.NetCDF4DataStore(nc))


url = "https://github.com/ioos/ioos_qc/raw/master/docs/source/examples"
fname = f"{url}/pco2_netcdf_example.nc#mode=bytes"

pco2 = open_from_https(fname)
pco2

Plot the raw data.

In [ ]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot


data = pco2["pco2_seawater"]
t = np.array(pco2["time"])
x = np.array(data)

p1 = figure(x_axis_type="datetime", title="pco2_seawater")
p1.grid.grid_line_alpha = 0.3
p1.xaxis.axis_label = "Time"
p1.yaxis.axis_label = data.units
p1.line(t, x)

show(gridplot([[p1]], width=800, height=400))

# QC Configuration

Here we define the generic config object for multiple QARTOD tests, plus the aggregate/rollup flag.

The key "pco2_seawater" indicates which variable in the netcdf file this config should run against.

In [ ]:
from ioos_qc.config import Config


config = {
    "pco2_seawater": {
        "qartod": {
            "gross_range_test": {"suspect_span": [200, 2400], "fail_span": [0, 3000]},
            "spike_test": {"suspect_threshold": 500, "fail_threshold": 1000},
            "location_test": {"bbox": [-124.5, 44, -123.5, 45]},
            "flat_line_test": {
                "tolerance": 1,
                "suspect_threshold": 3600,
                "fail_threshold": 86400,
            },
            "aggregate": {},
        }
    }
}


config = """
  contexts:
    - streams:
        pco2_seawater:
          qartod:
              gross_range_test:
                suspect_span: [200, 2400]
                fail_span: [0, 3000]
              spike_test:
                suspect_threshold: 500
                fail_threshold: 1000
              location_test:
                bbox: [-124.5, 44, -123.5, 45]
              flat_line_test:
                tolerance: 1
                suspect_threshold: 3600
                fail_threshold: 86400
"""

c = Config(config)

# Option A: Manually run tests and store results

Store test configurations externally, pass your configuration and netcdf file to `ioos_qc`, and manually update netcdf variables with results of the test.


Note: For tests that need tinp, zinp, etc, use args to define the t, x, y, z dimensions.
In this case, we need latitude and longitude for the location test.

In [ ]:
from ioos_qc.qartod import aggregate
from ioos_qc.streams import XarrayStream
from ioos_qc.results import collect_results, CollectedResult

qc = XarrayStream(pco2, lon="lon", lat="lat")

# Store as a list to run QC now
runner = list(qc.run(c))

results = collect_results(runner, how='list')

agg = CollectedResult(
    stream_id='',
    package='qartod',
    test='qc_rollup',
    function=aggregate,
    results=aggregate(results),
    tinp=qc.time(),
    data=data
)
results.append(agg)
results

In [ ]:
plot_ncresults(pco2, "pco2_seawater", results, "pCO2 seawater", "gross_range_test")

In [ ]:
plot_ncresults(pco2, "pco2_seawater", results, "pCO2 seawater", "spike_test")

In [ ]:
plot_ncresults(pco2, "pco2_seawater", results, "pCO2 seawater", "flat_line_test")

In [ ]:
plot_ncresults(pco2, "pco2_seawater", results, "pCO2 seawater", "location_test")

In [ ]:
# To see overall results, use the aggregate test
plot_ncresults(pco2, "", results, "pCO2 seawater", "qc_rollup")

In [ ]:
# Store results manually
# This is just a simple example and stores the aggregate test flag as a variable.
# You can expand upon this, or use the ioos_qc library to store the results for you (see subsequent examples)
# and use xarray's to_xxx methods to serialize results to whichever format you prefer

agg_da = xr.DataArray(agg.results, {}, ('time',))
output_xds = pco2.assign(
    qartod_aggregate=agg_da
)

output_xds.qartod_aggregate

# Option B

Store test configurations externally, then pass your configuration and netcdf file to `ioos_qc`, and let it run tests and update the file with results

In [ ]:
# Using the CFNetCDFStore Store we can serialize our results back to a CF compliant netCDF file easily
from ioos_qc.stores import CFNetCDFStore
from pocean.dsg import OrthogonalMultidimensionalTimeseries

# We use the `results` from Option A so we don't repeat ourselves.
store = CFNetCDFStore(runner)

outfile_b = os.path.join(tempfile.gettempdir(), "out_b.nc")
if os.path.exists(outfile_b):
    os.remove(outfile_b)

qc_all = store.save(
    # The netCDF file to export OR append to
    outfile_b,
    # The DSG class to save the results as
    OrthogonalMultidimensionalTimeseries,
    # The QC config that was run
    c,
    # Should we write the data or just metadata? Defaults to false
    write_data=True,
    # Compute a total aggregate?
    compute_aggregate=True,
    # Any kwargs to pass to the DSG class
    dsg_kwargs=dict(
        reduce_dims=True,  # Remove dimensions of size 1
        unlimited=False,  # Don't make the record dimension unlimited
        unique_dims=True,  # Support loading into xarray
    ),
)

In [ ]:
# Explore results: qc test variables are named [variable_name]_qartod_[test_name]
out_b = xr.open_dataset(outfile_b)
out_b

In [ ]:
# Gross range test
# Note how the config used is stored in the ioos_qc_* variables
out_b["pco2_seawater_qartod_gross_range_test"]

In [ ]:
# Aggregate/rollup flag
out_b["qartod_qc_rollup"]

# Option C

Store test configurations in your netcdf file, then pass that file to `ioos_qc` and let it run tests and update the file with results.

In the example above, we used the library to store results and config in the netcdf file itself. At this point, we can load that same file and run tests again, without having to re-define config. This is very powerful!

In [ ]:
# Create a copy of the output from B
infile_c = os.path.join(tempfile.gettempdir(), "in_c.nc")
shutil.copy(outfile_b, infile_c)

# Load this file into the Config object
input_c = xr.open_dataset(infile_c)
qc_config_c = Config(input_c)

In [ ]:
# The QC functions that will be run are extracted from the netCDF attributes
qc_config_c.calls

In [ ]:
# We can use that Config just like any other config object.
# Here we will re-run the netCDF file with the config extracted from the same netCDF file

# Setup input stream from file
xrs_c = XarrayStream(input_c, lon="lon", lat="lat")

# Setup config run
runner_c = list(xrs_c.run(qc_config_c))

# Collect QC run results
results = collect_results(runner_c, how='list')

# Compute Aggregate
agg = CollectedResult(
    stream_id='',
    package='qartod',
    test='qc_rollup',
    function=aggregate,
    results=aggregate(results),
    tinp=xrs_c.time(),
    data=data
)
results.append(agg)

In [ ]:
store = CFNetCDFStore(runner)

outfile_c = os.path.join(tempfile.gettempdir(), "out_c.nc")
if os.path.exists(outfile_c):
    os.remove(outfile_c)

qc_all = store.save(
    # The netCDF file to export OR append to
    outfile_c,
    # The DSG class to save the results as
    OrthogonalMultidimensionalTimeseries,
    # The QC config that was run
    c,
    # Should we write the data or just metadata? Defaults to false
    write_data=True,
    # Compute a total aggregate?
    compute_aggregate=True,
    # Any kwargs to pass to the DSG class
    dsg_kwargs=dict(
        reduce_dims=True,  # Remove dimensions of size 1
        unlimited=False,  # Don't make the record dimension unlimited
        unique_dims=True,  # Support loading into xarray
    ),
)

In [ ]:
# Explore results: qc test variables are named [variable_name]_qartod_[test_name]
out_c = xr.open_dataset(outfile_c)
out_c

In [ ]:
# Aggregate/rollup flag
out_c["qartod_qc_rollup"]